In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import scale
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from tensorflow.keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

In [ ]:
data = pd.read_csv("PHM07.csv")
features = scale(data.iloc[:,2:26]) # select required columns and scale them
labels = data.iloc[:,26] # select RUL
# print("done right")

In [ ]:
def windows(nrows, size):
    start,step = 0, 2
    while start < nrows:
        yield start, start + size
        start += step

def segment_signal(features,labels,window_size = 15):
    segments = np.empty((0,window_size))
    segment_labels = np.empty((0))
    nrows = len(features)
    for (start, end) in windows(nrows,window_size):
        if(len(data.iloc[start:end]) == window_size):
            segment = features[start:end].T  #Transpose to get segment of size 24 x 15 
            label = labels[(end-1)]
            segments = np.vstack([segments,segment]) 
            segment_labels = np.append(segment_labels,label)
    segments = segments.reshape(-1,24,window_size,1) # number of features  = 24 
    segment_labels = segment_labels.reshape(-1,1)
    return segments,segment_labels

In [ ]:
segments, labels = segment_signal(features,labels)



In [ ]:
train_test_split = np.random.rand(len(segments)) < 0.70
train_x = segments[train_test_split]
train_y = labels[train_test_split]
test_x = segments[~train_test_split]
test_y = labels[~train_test_split]


In [ ]:
print ("number of training examples = " + str(train_x.shape[0]))
print ("number of test examples = " + str(test_x.shape[0]))
print ("X_train shape: " + str(train_x.shape))
print ("Y_train shape: " + str(train_y.shape))
print ("X_test shape: " + str(test_x.shape))
print ("Y_test shape: " + str(test_y.shape))

number of training examples = 7416
number of test examples = 3155
X_train shape: (7416, 24, 15, 1)
Y_train shape: (7416, 1)
X_test shape: (3155, 24, 15, 1)
Y_test shape: (3155, 1)


In [ ]:
def HappyModel(input_shape):
   
    X_input = Input(input_shape)
    
    # Adding padding of 3.
    Xa = ZeroPadding2D((3, 3))(X_input)
    # Convolutional, F=7, S=1.
    Xb = Conv2D(32, (7, 7), strides=(1, 1), name='conv0')(Xa)
    # Batch Normalization after convolution. Axis=3, normalize over the #channels.
    Xc = BatchNormalization(axis=3, name='bn0')(Xb)
    # ReLU
    Xd = Activation('relu')(Xc)

    # Max Pooling F=2.
    Xe = MaxPooling2D((2, 2), name='max_pool')(Xd)

    # Flatten output
    # Just one neuron for this layer.
    Xf = Flatten()(Xe)
    Xg = Dense(1, activation='relu', name='fc')(Xf)

    # Model
    model = Model(inputs=X_input, outputs=Xg, name='HappyModel')

    return model

In [ ]:
happyModel = HappyModel(input_shape=(24,15,1))

In [ ]:
happyModel.compile(optimizer='Adam', loss='mse', metrics=['accuracy'])


In [ ]:
happyModel.fit(x=train_x, y=train_y, batch_size=60, epochs=25)


Epoch 1/25
7416/7416 [==============================] - 3s 438us/step - loss: 3119.9509 - accuracy: 0.0063
Epoch 2/25
7416/7416 [==============================] - 3s 402us/step - loss: 3059.5492 - accuracy: 0.0073
Epoch 3/25
7416/7416 [==============================] - 3s 404us/step - loss: 3011.2898 - accuracy: 0.0055
Epoch 4/25
7416/7416 [==============================] - 3s 402us/step - loss: 2953.3792 - accuracy: 0.0059
Epoch 5/25
7416/7416 [==============================] - 3s 445us/step - loss: 2918.9938 - accuracy: 0.0067
Epoch 6/25
7416/7416 [==============================] - 4s 472us/step - loss: 2849.5804 - accuracy: 0.0082
Epoch 7/25
7416/7416 [==============================] - 3s 466us/step - loss: 2834.6507 - accuracy: 0.0082
Epoch 8/25
7416/7416 [==============================] - 4s 473us/step - loss: 2790.5128 - accuracy: 0.0085
Epoch 9/25
7416/7416 [==============================] - 3s 472us/step - loss: 2736.2948 - accuracy: 0.0092
Epoch 10/25
7416/7416 [==============

In [ ]:
# x_input = array([[80, 85], [90, 95], [100, 105]])
# x_input = x_input.reshape((1, n_steps, n_features))
yhat = happyModel.predict(test_x, verbose=0)
print(test_x)
print(yhat)

In [ ]:
p=happyModel.get_layer(index=1)
print(type(p))
print(p.get_weights())

<class 'keras.layers.convolutional.ZeroPadding2D'>
[]


In [ ]:
preds = happyModel.evaluate(x=test_x, y=test_y, batch_size=150)

3155/3155 [==============================] - 1s 159us/step


In [ ]:
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]*100))


Test Accuracy = 0.9508715942502022
